# Baseline solution

Here I will implement a baseline solution: simple "toxic" words removal.

The list of toxic words is taken from https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/blob/master/en.

In [1]:
with open('../data/external/en.txt') as file:
    badwords = [line.rstrip() for line in file]

In [2]:
badwords[0:10]

['2g1c',
 '2 girls 1 cup',
 'acrotomophilia',
 'alabama hot pocket',
 'alaskan pipeline',
 'anal',
 'anilingus',
 'anus',
 'apeshit',
 'arsehole']

Do not need training here.

In [3]:
import pandas as pd

test_data = pd.read_csv('../data/interim/test.csv')
test_data.head()

,reference,translation
0,Fucking A your mom likes lan.,my mom loves you.
1,We'll be fucking pariahs.,we're going to be completely unnerved.
2,"I'm done, Live Dead.","I'm through, Dead Meat."
3,What is this place? A fucking vampire secret h...,that's a secret vampire headquarters.
4,Just a silly dream and nothing more,# Just a silky dream and nothing more


## Toxic words removal

In [4]:
import re
from tqdm import tqdm

def remove_toxic_words(texts):
    clean_texts = []
    
    for i in tqdm(range(len(texts))):
        regex = re.compile(r"\b" + r"\b|\b".join(map(re.escape, badwords)) + r"\b", re.IGNORECASE)
        clean_texts.append(regex.sub("", texts[i]))

    return clean_texts

In [5]:
pred = remove_toxic_words(list(test_data.reference))

100%|██████████████████████████████████████████████████████████████████████████| 85352/85352 [01:02<00:00, 1364.69it/s]


## Algorithm performance

### Similarity

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
cosine_sims = []

for i in tqdm(range(len(test_data))):
    texts = [test_data.reference[i], pred[i]]
    vector_matrix = count_vectorizer.fit_transform(texts)

    cosine_sims.append(cosine_similarity(vector_matrix)[0][1])
    
print("Average similarity:", sum(cosine_sims) / len(cosine_sims))

100%|███████████████████████████████████████████████████████████████████████████| 85352/85352 [02:30<00:00, 566.26it/s]


Average similarity: 0.9780516129612405


The similarity is very high because we don't add other words to the text, only remove toxic ones. But the logic of some of the sentences is broken.

## Examples of work

In [25]:
for i in [8, 58, 650]: 
    print("Reference:", test_data.reference[i])
    print("Detoxifyed:", pred[i])
    print()

Reference: Doesn't anybody in this town speak in complete fucking sentences anymore?
Detoxifyed: Doesn't anybody in this town speak in complete  sentences anymore?

Reference: you even tried to wipe your butt off.
Detoxifyed: you even tried to wipe your  off.

Reference: My eyes are fucked up.
Detoxifyed: My eyes are fucked up.



What can be seen from some examples:
1. The first example shows the successful work of the remover. This is the case when a toxic word can be simply removed without losing the meaning and logic of the sentence.
2. The second example just cut out a word and now the sentence has lost its logical meaning.
3. The remover is limited to only a list of bad words, which means that not all toxic words will be removed.

## Conclusions:
The idea of this algorithm is very simple, so the obtained results are expected.

The algorithm did not significantly reduce the toxicity of sentences due to the limited word list and the inability to recognize toxic context. Similarity to the original sentences remains high, but the sentences themselves often lose grammatical meaning.